In [ ]:
pip install -U sentence-transformers

     |████████████████████████████████| 81kB 3.5MB/s 
     |████████████████████████████████| 2.3MB 8.2MB/s 
     |████████████████████████████████| 1.2MB 33.1MB/s 
     |████████████████████████████████| 3.3MB 32.8MB/s 
     |████████████████████████████████| 901kB 34.6MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.1.1-cp37-none-any.whl size=123338 sha256=5c799efa2504a17e727086d9fd80c5680de3f4620e5f0f2492047a5d01532ad1
  Stored in directory: /root/.cache/pip/wheels/5e/89/29/45e45adc162b50f97f71801e8b07947c9cfe2b3ae7dbf37896
Successfully built sentence-transformers


In [ ]:
import scipy
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('bert-base-nli-mean-tokens')
sentences = ['This framework generates embeddings for each input sentence',
    'Sentences are passed as a list of string.', 
    'The quick brown fox jumps over the lazy dog.']
sentence_embeddings = model.encode(sentences)

print("Sentence embeddings:")
print(sentence_embeddings.shape)

Some weights of the model checkpoint at /root/.cache/torch/sentence_transformers/sbert.net_models_bert-base-nli-mean-tokens/0_BERT were not used when initializing BertModel: ['classifier.weight', 'classifier.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Sentence embeddings:
(3, 768)


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
data=pd.read_csv('/content/drugsComTrain_raw.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,2,3,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
data.head()

,uniqueID,drugName,condition,review,rating,date,usefulCount
0,206461.0,Valsartan,Left Ventricular Dysfunction,"""It has no side effect, I take it in combinati...",9.0,20-May-12,27.0
1,95260.0,Guanfacine,ADHD,"""My son is halfway through his fourth week of ...",8.0,27-Apr-10,192.0
2,92703.0,Lybrel,Birth Control,"""I used to take another oral contraceptive, wh...",5.0,14-Dec-09,17.0
3,138000.0,Ortho Evra,Birth Control,"""This is my first time using any form of birth...",8.0,03-Nov-15,10.0
4,35696.0,Buprenorphine / naloxone,Opiate Dependence,"""Suboxone has completely turned my life around...",9.0,27-Nov-16,37.0


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 161296 entries, 0 to 161295
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   uniqueID     27434 non-null  float64
 1   drugName     27434 non-null  object 
 2   condition    27434 non-null  object 
 3   review       27434 non-null  object 
 4   rating       27434 non-null  float64
 5   date         27434 non-null  object 
 6   usefulCount  27434 non-null  float64
dtypes: float64(3), object(4)
memory usage: 8.6+ MB


In [ ]:
data.dropna(inplace=True,axis=0)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27434 entries, 0 to 27433
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   uniqueID     27434 non-null  float64
 1   drugName     27434 non-null  object 
 2   condition    27434 non-null  object 
 3   review       27434 non-null  object 
 4   rating       27434 non-null  float64
 5   date         27434 non-null  object 
 6   usefulCount  27434 non-null  float64
dtypes: float64(3), object(4)
memory usage: 1.7+ MB


In [ ]:
complaints=list(data['review'])


In [ ]:
#GPU
import time
s=time.time()
complaint_embeddings = model.encode(complaints)
time.time()-s

97.05251502990723

In [ ]:
# import time
# s=time.time()
# complaint_embeddings = model.encode(complaints[:100])
# time.time()-s

60.92678928375244

In [ ]:
complaint_embeddings.shape

(27434, 768)

In [ ]:
problem='Chest feels heavy and difficulty in breathing, I keep coughing. Sweating alot'

In [ ]:
problem_embedding=model.encode(problem)

In [ ]:
complaint_embeddings[:2,:].shape

(2, 768)

In [ ]:
np.round(1-scipy.spatial.distance.cdist([problem_embedding], complaint_embeddings[:4,:], "cosine"),2)
#round((1-scipy.spatial.distance.cdist([problem_embedding], [complaint_embeddings[0]], "cosine")[0][0]),2)

array([[0.13, 0.34, 0.28, 0.29]])

In [ ]:
similarities[:5]


array([0.13, 0.35, 0.28, 0.28, 0.3 ])

In [ ]:
similarities.flatten()[:5]

array([0.13, 0.35, 0.28, 0.28, 0.3 ])

In [ ]:
similarities=np.round(1-scipy.spatial.distance.cdist([problem_embedding], complaint_embeddings, "cosine"),2)
similarities=similarities.flatten()
similarities_desc=np.sort(similarities)[::-1]
#np.argsort(similarities)[::-1]
indexes=np.array([i for i in range(complaint_embeddings.shape[0])])
best_matches=indexes[np.argsort(similarities)[::-1]]

In [ ]:
topn=3
print(problem)
print()
for i in range(topn):
  print(data['review'][best_matches[i]],data['condition'][best_matches[i]],similarities[best_matches[i]])
  print()
  print()

  

Chest feels heavy and difficulty in breathing, I keep coughing. Sweating alot

"Dizziness, tiredness, runny nose, sore throat, cough, diarrhea, generally feel bad." High Cholesterol 0.8


"I have to take more and more. It wears off fast. I also get constipated and itch like crazy from it!" Restless Legs Syndrome 0.79


"Was given this for pain, had bad shaking, very anxious and irratable. Also dizzy and nauseated." Pain 0.78




In [ ]:
from sys import getsizeof
round(getsizeof(complaint_embeddings) / 1024 / 1024,2)

80.37